In [ ]:
%mavenRepo s3 https://joule.s3.amazonaws.com/repo

In [ ]:
%maven software.amazon.ai:joule-api:0.2.0-SNAPSHOT
%maven org.apache.mxnet:mxnet-joule:0.2.0-SNAPSHOT
%maven org.slf4j:slf4j-api:1.7.26
%maven net.java.dev.jna:jna:5.3.0
// %maven org.apache.mxnet:mxnet-native-mkl:jar:osx-x86_64:1.5.0-SNAPSHOT

In [ ]:
%%loadFromPOM
  <repositories>
    <repository>
      <id>joule</id>
      <url>https://joule.s3.amazonaws.com/repo</url>
    </repository>
  </repositories>

  <dependencies>
    <dependency>
      <groupId>org.apache.mxnet</groupId>
      <artifactId>mxnet-native-mkl</artifactId>
      <version>1.5.0-SNAPSHOT</version>
      <classifier>osx-x86_64</classifier>
    </dependency>
  </dependencies>


In [ ]:
import java.awt.image.*;
import java.nio.file.*;
import java.util.*;
import software.amazon.ai.*;
import software.amazon.ai.inference.*;
import software.amazon.ai.ndarray.*;
import software.amazon.ai.ndarray.index.*;
import software.amazon.ai.modality.*;
import software.amazon.ai.modality.cv.*;
import software.amazon.ai.util.*;


In [ ]:
String userHome = System.getProperty("user.home");
Path modelDir = Paths.get(userHome, ".model_zoo/squeezenet_v1.1");
Path imageFile = Paths.get("../examples/src/test/resources/kitten.jpg").toRealPath();
String modelName = "squeezenet_v1.1";

modelDir

In [ ]:
BufferedImage img = Images.loadImageFromFile(imageFile);
img

In [ ]:
Model model = Model.loadModel(modelDir, modelName);
List<String> synset = model.getArtifact("synset.txt", Utils::readLines);

In [ ]:
// Resize image to match the size that get trained for the model

BufferedImage resizedImg = Images.resizeImage(img, 224, 224);

In [ ]:
DefaultImageTranslator translator = new DefaultImageTranslator();
Predictor<BufferedImage, NDList> predictor = Predictor.newInstance(model, translator);
NDList ndList = predictor.predict(resizedImg);

In [ ]:
// Post processing

NDArray array = ndList.get(0).get(0);
NDArray sorted = array.argsort(-1, false);
NDArray top = sorted.get(0);

top

In [ ]:
int index = sorted.getInt(0);

In [ ]:
String className = synset.get(index);
float probability = array.getFloat(index);

className + ": " + probability
